# Workstation Heartbeat and Live Data

This example shows how in a separate thread from the term the HeartBeat can be queried with Python. The heartbeat is queried in a separate thread once a second. The HeartBeat represents how many milliseconds it has been since the term has received something from the Thales.

This example also receives the live data. To receive the online display data, the Zahner online display must be switched off.
To do this, the following parameter must be adapted in the file C:/FLINK/usb.ini EnableODisplay=off.

In [1]:
import sys
from thales_remote.connection import ThalesRemoteConnection
from thales_remote.script_wrapper import PotentiostatMode,ThalesRemoteScriptWrapper
import time
import threading

It is not a clean solution that the connections are global variables.
This was solved for a simpler example with global variables.

In [2]:
zenniumConnection = None
zahnerZennium = None
zenniumConnectionLiveData = None

keepThreadRunning = True

# Watch Thread
The following function is used as a thread, in which the HearBeat is queried once a second. The HeartBeat time varies, for example, if EIS is measured at low frequencies, then this time is increased.

The HeartBeat is queried once per second. A timeout of 2 seconds is used to query the HeartBeat. This ensures that the Term responds within 2 seconds, otherwise it can be assumed that the Term software has crashed.

In [3]:
def watchThreadFunction():
    global keepThreadRunning
    global zenniumConnection
    global zahnerZennium
    
    print("watch thread started")
    while keepThreadRunning:
        time.sleep(1)
        try:
            beat = zahnerZennium.getWorkstationHeartBeat(2)
        except:
            print("term error watch thread")
            keepThreadRunning = False
        else:
            print("Heartbeat: " + str(beat) + " ms")
    
    print("watch thread left")
    return

# Live Data Thread
The following function is used as a thread which receives the live data instead of the online display.

Only relevant packet types are output to the console. The relevant types are written as comments in the source code.

In [4]:
def liveDataThreadFunction():
    global keepThreadRunning
    global zenniumConnectionLiveData
    
    print("live thread started")
    while keepThreadRunning:
        try:
            data = zenniumConnectionLiveData.waitForBinaryTelegram()
            packetId = data[0]
            data = data[1:]
            '''
            Type:
            1 = Init measurement begin
            2 = Measurement end
            4 = Measurement data names
            5 = Measurement data units
            6 = ASCII data
            '''
            if packetId in [1,2,4,5,6]:
                print(data.decode("ASCII"))
        except:
            '''
            The connection to the term has an error or the socket has been closed.
            '''
            print("term error live thread")
            keepThreadRunning = False
    
    print("live thread left")
    return

# Main Program Sequence

In the main program flow, the first thing that happens is that an additional connection to the term is established with the name "Logging". The live data comes via this connection.

Then the thread is started, which receives the data.

In [5]:
if __name__ == "__main__":
    zenniumConnectionLiveData = ThalesRemoteConnection()
    zenniumConnectionLiveData.connectToTerm("localhost", "Logging")
        
    
    liveThread = threading.Thread(target=liveDataThreadFunction)
    liveThread.start()

live thread started


After the connection with the live data, the nominal connection is established, which sends the commands for measurement.

In addition, the hideWindow() method hides the Thales window to protect it from unwanted operation via GUI when controlled remotely.

In [6]:
    zenniumConnection = ThalesRemoteConnection()
    zenniumConnection.connectToTerm("localhost", "ScriptRemote")
        
    zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)
    zahnerZennium.forceThalesIntoRemoteScript()
    zahnerZennium.hideWindow()
    
    zahnerZennium.calibrateOffsets()

'OK\r'

The watch thread uses the command interface to the Thales, so it is started after initializing this connection.

In [7]:
    watchThread = threading.Thread(target=watchThreadFunction)
    watchThread.start()
    
    zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
    zahnerZennium.setAmplitude(10e-3)
    zahnerZennium.setPotential(0)
    zahnerZennium.setLowerFrequencyLimit(750)
    zahnerZennium.setStartFrequency(1000)
    zahnerZennium.setUpperFrequencyLimit(1500)
    zahnerZennium.setLowerNumberOfPeriods(2)
    zahnerZennium.setLowerStepsPerDecade(2)
    zahnerZennium.setUpperNumberOfPeriods(2)
    zahnerZennium.setUpperStepsPerDecade(20)
    zahnerZennium.setScanDirection("startToMax")
    zahnerZennium.setScanStrategy("single")
    
    zahnerZennium.enablePotentiostat()
    
    
    zahnerZennium.setFrequency(1)
    zahnerZennium.setAmplitude(10e-3)
    zahnerZennium.setNumberOfPeriods(3)
    
    print("measurement start")
    
    zahnerZennium.measureEIS()
    for i in range(20):
        zahnerZennium.getPotential()
        zahnerZennium.setPotential(0)

    print("measurement end")
        
    zahnerZennium.disablePotentiostat()

watch thread started
measurement start
Heartbeat: 281.0 ms
3,Impedance Spectroscopy
frequency,impedance,phase,time,significance,voltage,current,
Hz,Ohm,rad,s, ,V,A,
Heartbeat: 172.0 ms
Heartbeat: 687.0 ms
Heartbeat: 15.0 ms
Heartbeat: 469.0 ms
 1.00940e+03, 1.00542e-02, 3.08309e-04, 0.00000e+00, 1.00000e+00, 5.74984e-06, 1.42959e-03,
 1.11450e+03, 1.00589e-02, 4.00650e-04, 1.85350e+00, 1.00000e+00, 5.74984e-06, 1.42959e-03,
Heartbeat: 63.0 ms
 1.23050e+03, 1.00501e-02, 8.81199e-05, 2.21900e+00, 1.00000e+00, 5.74984e-06, 1.42959e-03,
 1.35860e+03, 1.00547e-02, 5.45079e-04, 2.65000e+00, 1.00000e+00, 5.74984e-06, 1.42959e-03,
Heartbeat: 266.0 ms
 1.50000e+03, 1.00534e-02, 2.23606e-06, 3.01250e+00, 1.00000e+00, 5.74984e-06, 1.42959e-03,
 1.35860e+03, 1.00557e-02, 3.30657e-04, 3.37450e+00, 1.00000e+00, 5.74984e-06, 1.42959e-03,
 1.23050e+03, 1.00567e-02, 4.46807e-04, 3.73800e+00, 1.00000e+00, 5.74984e-06, 1.42959e-03,
Heartbeat: 172.0 ms
 1.11450e+03, 1.00546e-02, 7.31679e-05, 4.10400e+00, 

'OK\r'

Closing the threads and then waiting until they are closed.

Before the connection is disconnected, the Thales window is displayed again.

In [8]:
    print("set thread kill flag")
    keepThreadRunning = False
    
    print("disconnect connections")
    zahnerZennium.showWindow()
    zenniumConnection.disconnectFromTerm()
    zenniumConnectionLiveData.disconnectFromTerm()
    
    print("join the threads")
    liveThread.join()
    watchThread.join()
    
    print("finish")

set thread kill flag
disconnect connections
term error watch thread
watch thread left
term error live thread
live thread left
join the threads
finish
